In [5]:
import pandas as pd
import numpy as np
import re
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import random
from collections import Counter
import pickle

In [17]:
user_movie_pivot = pd.read_csv('../dataset/user_movie_pivot.zip', compression='zip', header=0, sep=',')
movie_list = user_movie_pivot.columns.tolist()

In [20]:
user_movie_pivot

,userId,2,3,5,6,11,12,13,14,15,...,132961,133365,134158,134569,134881,140174,142507,148652,158238,160718
0,U1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,U10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,U100,0.0,10.0,0.0,-10.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,U101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,U102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666,U95,0.0,0.0,0.0,10.0,10.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
667,U96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668,U97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
669,U98,0.0,0.0,-10.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# This functin returns movies names and ids. To use it, there should be a variable storing the id for the next recommending function.
def provide_movies_to_user(num):
    ran_id = random.sample(movie_list, num)
    ran_name = [movie_lookup_dict[ran] for ran in ran_id]
    return ran_name,ran_id

In [8]:
# Helper function 1 -- to exclude movies that users already seen from the recommendation list
def exclude_seen_movies(u_select):
    # get seen movie id from user feedback
    seen_id_list = [m for m,r in u_select if r!=0]
    seen_name_list = [movie_lookup_dict[x] for x in seen_id_list]
    return seen_name_list      

In [9]:
# Helper function 2 -- to get users with similar preferences for these 10 movies
def get_similar_users(u_select):
    # get movie id of user feedback
    p_id_list = [m for m,r in u_select if r>0]
    n_id_list = [m for m,r in u_select if r<0]

    # get index of movie for user feedback
    p_idx_list = [movie_list.index(x) for x in p_id_list]
    n_idx_list = [movie_list.index(x) for x in n_id_list]

    # create input array of same dimension for prediction
      # modify value of corresponding index to 10/-10
    user_fb_arr = np.zeros((1,2807))
    for idx1 in p_idx_list:
        user_fb_arr[0,idx1] = 10
    for idx2 in n_idx_list:
        user_fb_arr[0,idx2] = -10
    
    # find users who have similar preferences for these movies
    neigh_idx = model_nn.kneighbors(user_fb_arr, 3, return_distance=False)
    users_id = [] 
    for user_idx, val in enumerate(user_movie_pivot.index[neigh_idx][0]):
        users_id.append(val)
        #print((user_idx+1),". ",val)
    return users_id

In [10]:
# Helper function 3 -- To get movies liked by those similar users 
def get_sim_user_like(sim_user_idx):
    user_row = user_movie_pivot.loc[[sim_user_idx]].values.flatten().tolist()
    user_like_movie_idx = []
    i = 0
    for y in user_row:
        if y == 10:
            user_like_movie_idx.append(i)
        i += 1
    if len(user_like_movie_idx) > 0:
        user_like_movie_id = [movie_list[m] for m in user_like_movie_idx]
        user_like_movie_name = [movie_lookup_dict[n] for n in user_like_movie_id]
        return user_like_movie_name
    else:
        print('no similar user found')


In [11]:
# Recommendation function
def recommend_to_user(u_fb):

    feedback_list = list(zip(provide[1],u_fb))
    sim_user_ids = get_similar_users(feedback_list)
    movie_exclude = exclude_seen_movies(feedback_list)

    recommend_list = []
    for each in sim_user_ids:
        each_like = get_sim_user_like(each)
        each_like_cl = [mov for mov in each_like if mov not in movie_exclude]
        recommend_list.extend(each_like_cl)

    counter = Counter(recommend_list)
    toplist = counter.most_common(5)
    recommend_movies = [tl for tl,ct in toplist]
    return recommend_movies

In [28]:
# Call provider function to show 10 movies to start with and store the movie name, id pair in a global variable
read_dict = pd.read_csv('../dataset/movie_lookup_dict.csv').to_dict('list')
movie_lookup_dict = {}
for k, v in read_dict.items():
    movie_lookup_dict[k] = v[0]

In [50]:
provide = provide_movies_to_user(10)
provide

(['Rebecca',
  'Dr. Cyclops',
  'Devil in the Flesh',
  'Three Kings',
  'Germany Year Zero',
  'My Name Is Bruce',
  'What Women Want',
  '25th Hour',
  'Dogville',
  'The Common Man'],
 ['223',
  '3593',
  '45431',
  '6415',
  '8016',
  '1961',
  '3981',
  '1429',
  '553',
  '5602'])

In [31]:
model_nn = pickle.load(open('../dataset/cf_model.sav', 'rb'))

In [32]:
# Upon receiving user feedback (the input of this function), make recommendations
recommend_to_user([10,0,-10,10,0,10,0,0])

/Users/chloezhang/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/range.py:780: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  return super().__getitem__(key)


['Cars',
 'My Own Private Idaho',
 'Sleepless in Seattle',
 'Madagascar',
 'Ghost Rider']

In [48]:
random_movies = pd.read_csv("../temp.csv",header=None,index_col=False,).values.tolist()
random_movies

[['My Life Without Me',
  'Dracula vs. Frankenstein',
  'My Own Private Idaho',
  'Exodus',
  'Love Actually',
  'The Island',
  '3 Ninjas: High Noon at Mega Mountain',
  'The Mexican'],
 ['20', '3125', '468', '1941', '508', '1635', '32302', '6073']]

In [46]:
movies = provide_movies_to_user(8)
movies

(['Presumed Guilty',
  'The Outsiders',
  'Wag the Dog',
  "In My Father's Den",
  'The Terminal',
  'Beowulf & Grendel',
  'The Rapture',
  'Nausicaä of the Valley of the Wind'],
 ['58376', '227', '586', '1126', '594', '5471', '1411', '81'])

In [49]:
[item for sublist in random_movies for item in sublist]

['My Life Without Me',
 'Dracula vs. Frankenstein',
 'My Own Private Idaho',
 'Exodus',
 'Love Actually',
 'The Island',
 '3 Ninjas: High Noon at Mega Mountain',
 'The Mexican',
 '20',
 '3125',
 '468',
 '1941',
 '508',
 '1635',
 '32302',
 '6073']